<a href="https://colab.research.google.com/github/kleczekr/tolkenizer/blob/master/cleaning_data_air_travel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exploring and cleaning the air travel file
### Coursera Tableau project

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/US Monthly Air Passengers.csv')

In [3]:
df.shape

(392820, 17)

The data has 271336 lines, grouped in 17 columns.

In [4]:
df.columns

Index(['Sum_PASSENGERS', 'AIRLINE_ID', 'CARRIER_NAME', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM',
       'ORIGIN_COUNTRY', 'ORIGIN_COUNTRY_NAME', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEST_STATE_NM', 'DEST_COUNTRY', 'DEST_COUNTRY_NAME',
       'YEAR', 'MONTH'],
      dtype='object')

Let's count empty values:

In [5]:
df.isnull().sum()

Sum_PASSENGERS             0
AIRLINE_ID               448
CARRIER_NAME             448
ORIGIN                     0
ORIGIN_CITY_NAME           0
ORIGIN_STATE_ABR        9986
ORIGIN_STATE_NM         9986
ORIGIN_COUNTRY             0
ORIGIN_COUNTRY_NAME        0
DEST                       0
DEST_CITY_NAME             0
DEST_STATE_ABR         11429
DEST_STATE_NM          11429
DEST_COUNTRY               1
DEST_COUNTRY_NAME          1
YEAR                       1
MONTH                      1
dtype: int64

The following combinations of null values seem to be apparent: AIRLINE_ID with CARRIER_NAME, ORIGIN_STATE_ABR with ORIGIN_STATE_NM, DEST_STATE_ABR with DEST_STATE_NM. The following columns have each a single null value: DEST_CITY_NAME, DEST_COUNTRY, DEST_COUNTRY_NAME, YEAR and MONTH.

I will first deal with the pairs of clusters of missing values and see if I can detect any patterns among them.

In [6]:
df[df.AIRLINE_ID.isnull()].head()

,Sum_PASSENGERS,AIRLINE_ID,CARRIER_NAME,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,YEAR,MONTH
0,0,NaN,NaN,AEX,"Alexandria, LA",LA,Louisiana,US,United States,AEX,"Alexandria, LA",LA,Louisiana,US,United States,2015.0,3.0
1,0,NaN,NaN,AEX,"Alexandria, LA",LA,Louisiana,US,United States,AFW,"Dallas/Fort Worth, TX",TX,Texas,US,United States,2015.0,4.0
2,0,NaN,NaN,AEX,"Alexandria, LA",LA,Louisiana,US,United States,ATL,"Atlanta, GA",GA,Georgia,US,United States,2015.0,3.0
3,89,NaN,NaN,AEX,"Alexandria, LA",LA,Louisiana,US,United States,BOG,"Bogota, Colombia",NaN,NaN,CO,Colombia,2015.0,1.0
4,108,NaN,NaN,AEX,"Alexandria, LA",LA,Louisiana,US,United States,BOG,"Bogota, Colombia",NaN,NaN,CO,Colombia,2015.0,3.0


In [7]:
df[df.AIRLINE_ID.isnull()].tail()

,Sum_PASSENGERS,AIRLINE_ID,CARRIER_NAME,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,YEAR,MONTH
443,0,NaN,NaN,YIP,"Detroit, MI",MI,Michigan,US,United States,YIP,"Detroit, MI",MI,Michigan,US,United States,2000.0,6.0
444,174,NaN,NaN,YOW,"Ottawa, Canada",ON,Ontario,CA,Canada,LAS,"Las Vegas, NV",NV,Nevada,US,United States,2006.0,11.0
445,1416,NaN,NaN,YVR,"Vancouver, Canada",BC,British Columbia,CA,Canada,PDX,"Portland, OR",OR,Oregon,US,United States,2000.0,6.0
446,1274,NaN,NaN,YVR,"Vancouver, Canada",BC,British Columbia,CA,Canada,SLC,"Salt Lake City, UT",UT,Utah,US,United States,2000.0,6.0
447,1227,NaN,NaN,YYZ,"Toronto, Canada",ON,Ontario,CA,Canada,LAS,"Las Vegas, NV",NV,Nevada,US,United States,2006.0,11.0


I can't see a detectable pattern in the rows which lack the AIRLINE_ID and CARRIER_NAME. Still, since these columns are not the most important, I am not going to delete these rows---I will rather fill the null values there with the string 'undefined'.

In [8]:
df.AIRLINE_ID = df.AIRLINE_ID.fillna('undefined')
df.CARRIER_NAME = df.CARRIER_NAME.fillna('undefined')

Let's now turn to origin and destination states.

In [9]:
df[df.DEST_STATE_ABR.isnull()].head()

,Sum_PASSENGERS,AIRLINE_ID,CARRIER_NAME,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,YEAR,MONTH
3,89,undefined,undefined,AEX,"Alexandria, LA",LA,Louisiana,US,United States,BOG,"Bogota, Colombia",NaN,NaN,CO,Colombia,2015.0,1.0
4,108,undefined,undefined,AEX,"Alexandria, LA",LA,Louisiana,US,United States,BOG,"Bogota, Colombia",NaN,NaN,CO,Colombia,2015.0,3.0
5,83,undefined,undefined,AEX,"Alexandria, LA",LA,Louisiana,US,United States,BOG,"Bogota, Colombia",NaN,NaN,CO,Colombia,2015.0,4.0
27,90,undefined,undefined,AEX,"Alexandria, LA",LA,Louisiana,US,United States,GUA,"Guatemala City, Guatemala",NaN,NaN,GT,Guatemala,2015.0,1.0
28,217,undefined,undefined,AEX,"Alexandria, LA",LA,Louisiana,US,United States,GUA,"Guatemala City, Guatemala",NaN,NaN,GT,Guatemala,2015.0,2.0


It seems that the rows with missing DEST_STATE_ABR and DEST_STATE_NM are simply travels to countries other than United States.

In [10]:
df[df.ORIGIN_STATE_ABR.isnull()].head()

,Sum_PASSENGERS,AIRLINE_ID,CARRIER_NAME,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,YEAR,MONTH
70,787,undefined,undefined,AMS,"Amsterdam, Netherlands",NaN,NaN,NL,Netherlands,JFK,"New York, NY",NY,New York,US,United States,2009.0,5.0
71,1036,undefined,undefined,AMS,"Amsterdam, Netherlands",NaN,NaN,NL,Netherlands,JFK,"New York, NY",NY,New York,US,United States,2009.0,7.0
72,1026,undefined,undefined,ARN,"Stockholm, Sweden",NaN,NaN,SE,Sweden,MIA,"Miami, FL",FL,Florida,US,United States,2000.0,1.0
73,703,undefined,undefined,ARN,"Stockholm, Sweden",NaN,NaN,SE,Sweden,MIA,"Miami, FL",FL,Florida,US,United States,2000.0,2.0
74,707,undefined,undefined,ARN,"Stockholm, Sweden",NaN,NaN,SE,Sweden,MIA,"Miami, FL",FL,Florida,US,United States,2000.0,3.0


Likewise, the rows with null values in ORIGIN_STATE_ABR and ORIGIN_STATE_NM are the flights which originate outside of United States.

It makes sense to group all these rows by filling the missing values as 'Non-USA'.

In [11]:
df.ORIGIN_STATE_ABR = df.ORIGIN_STATE_ABR.fillna('Non-USA')
df.ORIGIN_STATE_NM = df.ORIGIN_STATE_NM.fillna('Non-USA')
df.DEST_STATE_ABR = df.DEST_STATE_ABR.fillna('Non-USA')
df.DEST_STATE_NM = df.DEST_STATE_NM.fillna('Non-USA')


Let's now consider the rows with single missing values.

In [16]:
df[df.DEST_COUNTRY_NAME.isnull()].head()

,Sum_PASSENGERS,AIRLINE_ID,CARRIER_NAME,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_COUNTRY,ORIGIN_COUNTRY_NAME,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,DEST_COUNTRY,DEST_COUNTRY_NAME,YEAR,MONTH
392819,8707,19393,Southwest Airlines Co.,LAS,"Las Vegas, NV",NV,Nevada,US,United States,GEG,"Spokane, WA",WA,Washingto,NaN,NaN,NaN,NaN


It seems that there was a single flight with all these columns missing---the Southwest Airlines Co. originating in Dallas. As this row lacks even date of the flight, it only makes sense to remove it.

In [17]:
df = df[df.DEST_COUNTRY_NAME.notna()]

Finally, let's take a look if we have any missing values left:

In [18]:
df.isnull().sum()

Sum_PASSENGERS         0
AIRLINE_ID             0
CARRIER_NAME           0
ORIGIN                 0
ORIGIN_CITY_NAME       0
ORIGIN_STATE_ABR       0
ORIGIN_STATE_NM        0
ORIGIN_COUNTRY         0
ORIGIN_COUNTRY_NAME    0
DEST                   0
DEST_CITY_NAME         0
DEST_STATE_ABR         0
DEST_STATE_NM          0
DEST_COUNTRY           0
DEST_COUNTRY_NAME      0
YEAR                   0
MONTH                  0
dtype: int64

Success!

In [15]:
from google.colab import files
df.to_csv('monthly_air_passengers.csv', index=False)